In [1]:
# this is a LLM agent that uses aspect based sentiment analysis

In [2]:
'''
New tool ideas

- SQL Query Generator

- SQL Query initiator

- Database Structure Analysis

- Data visulization

- Trend detection agent
'''

'\nNew tool ideas\n\n- SQL Query Generator\n\n- SQL Query initiator\n\n- Database Structure Analysis\n\n- Data visulization\n\n- Trend detection agent\n'

In [3]:
!pip install pyabsa[dev] -U
!pip install torch transformers datasets google-search-results
!pip install python-dotenv
!pip install openai

In [4]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env


In [5]:
# import relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import altair as alt
import os
import time
import json

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, LlamaTokenizer, LlamaForCausalLM
from pyabsa import AspectTermExtraction as ATEPC, available_checkpoints
from serpapi import GoogleSearch
from dotenv import load_dotenv
from openai import OpenAI

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/spacy/cli/_util.py:23: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string
/usr/local/lib/python3.12/dist-packages/weasel/util/config.py:8: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string


[2025-10-23 08:24:19] (2.4.2) PyABSA(2.4.2): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






In [6]:
load_dotenv()
SERPAPIKEY = os.getenv("SERPAPIKEY")
client = OpenAI(api_key=os.getenv("OPENAIAPIKEY"))
CONFIDENCETHRESHOLD = 0.8

In [8]:
modelName = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(modelName)
model = AutoModelForSequenceClassification.from_pretrained(modelName)

llmModel = "gpt-4o-mini"

sentimentPipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x79aee772b000>


added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x79aee0e417f0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x79aee0e41a90>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x79aee0e42120>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x79aee0e42270>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x79aee0e42190>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x79aee0e42350>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x79aee0e42430>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x79aee0e42510>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x79aee0e42660>
Device set to use cuda:0


---
Sentiment & API
---

In [9]:
# This is an example of basic sentiment analysis

text = "I love this ice-cream"

print(sentimentPipeline(text))

[{'label': 'Positive', 'score': 0.9802641868591309}]


In [10]:
# This is an example of aspect based sentiment analysis

aspectIceCream = "ice-cream"
aspectCone = "cone"
text = "I love this ice-cream, but the cone is terrible"

inputIce = f"{text} [SEP] {aspectIceCream}"
inputCone = f"{text} [SEP] {aspectCone}"

print("Ice cream sentiment: " + str(sentimentPipeline(inputIce)))
print("Cone sentiment: " + str(sentimentPipeline(inputCone)))

Ice cream sentiment: [{'label': 'Positive', 'score': 0.9964743256568909}]
Cone sentiment: [{'label': 'Negative', 'score': 0.9955949187278748}]


In [11]:
# We can also use something called pyabsa to handle the different aspects
# Update - This does not work as pyabsa cannot use the model im using

# aspects = ATEPCCheckpointManager.get_aspect_extractor(
#     checkpoint="yangheng/deberta-v3-base-absa-v1.1",
#     auto_device=True # This allows us to use a GPU if one is avaliabel
# )

# text = "I love this ice-cream, but the cone is terrible"

# results = aspects.extract_aspect(
#     inference_source=[text],
#     print_result=True
# )

In [12]:
# This is code directly from the "yangheng/deberta-v3-base-absa-v1.1" hugging face page that handles aspect extraction

def aspectExtraction(texts):
  aspect_extractor = ATEPC.AspectExtractor(
    'multilingual',
    auto_device=True,
    cal_perplexity=True
  )

  results = aspect_extractor.predict(
    texts,
    print_result=False,
    save_result=False,
    ignore_error=True,
    pred_sentiment=True
  )

  aspectSentiments = []

  for result in results:
    aspects = result['aspect']
    sentiments = result['sentiment']
    confidences = result['confidence']
    sentence = result['sentence']

    for i in range(len(aspects)):
      if confidences[i] > CONFIDENCETHRESHOLD:
        aspectSentiments.append({
            "aspect": aspects[i],
            "sentiment": sentiments[i],
            "sentence": sentence
        })

  return aspectSentiments


In [13]:
# print(aspectSentiments)

NameError: name 'aspectSentiments' is not defined

In [ ]:
# print(result)

In [14]:
# now I am going to attempt to use SerpAPI free tier to get all the google reviews from a certain company
SERPAPIKEY = "800f4e7deaecabcd22b118faadc9f33ac0a65583f5f4e9d1ef5cd9c808cb78ab"

# First we need to get the actual data_id of the location
location = "Swinburne University of Technology"

mapsParameters = {
  "engine": "google_maps",
  "q": location,
  "type": "search",
  "api_key": SERPAPIKEY
}

mapsSearch = GoogleSearch(mapsParameters)
mapsResults = mapsSearch.get_dict()

print(mapsResults)

dataId = mapsResults.get("local_results")[0].get("data_id")

{'search_metadata': {'id': '68f9e6f9dbb9ff9c2ded2e34', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/a9f0104e80a9f4f1/68f9e6f9dbb9ff9c2ded2e34.json', 'created_at': '2025-10-23 08:27:37 UTC', 'processed_at': '2025-10-23 08:27:37 UTC', 'google_maps_url': 'https://www.google.com/maps/search/Swinburne+University+of+Technology?hl=en', 'raw_html_file': 'https://serpapi.com/searches/a9f0104e80a9f4f1/68f9e6f9dbb9ff9c2ded2e34.html', 'total_time_taken': 0.55}, 'search_parameters': {'engine': 'google_maps', 'type': 'search', 'q': 'Swinburne University of Technology', 'google_domain': 'google.com', 'hl': 'en'}, 'search_information': {'local_results_state': 'Results for exact spelling', 'query_displayed': 'Swinburne University of Technology'}, 'local_results': [{'position': 1, 'title': 'Swinburne University of Technology', 'place_id': 'ChIJr1quazJC1moRafeISb1r6XU', 'data_id': '0x6ad642326bae5aaf:0x75e96bbd4988f769', 'data_cid': '8496440632737855337', 'reviews_link': 'https://s

In [15]:
# Then we need to extract the reviews from google

def getReviews(dataId):
  reviewParameters = {
      "engine": "google_maps_reviews",
      "data_id": dataId,
      "api_key": SERPAPIKEY,
      "hl": "en"
  }

  reviewSearch = GoogleSearch(reviewParameters)
  reviewResults = reviewSearch.get_dict()

  reviewTexts = []

  for review in reviewResults.get("reviews", []):
    reviewTexts.append(review["snippet"])

  return reviewTexts

In [16]:
for review in reviewResults.get("reviews", []):
  print(f"{review['snippet']}")
  print("-----------------------------------------------------------")

NameError: name 'reviewResults' is not defined

In [ ]:
reviewTexts = []

for review in reviewResults.get("reviews", []):
  reviewTexts.append(review["snippet"])

confidenceThreshold = 0.5

aspects = aspectExtraction(reviewTexts, confidenceThreshold)

for aspect in aspects:
  print(f"aspect: {aspect["aspect"]}")
  print(f"sentiment: {aspect["sentiment"]}")
  print("-----------------------------------------------------------")

In [ ]:
print(aspects)

---
Data Visulisation
---

In [17]:
def simplePlot(df: pd.DataFrame, x: str, y: str, kind: str = "line"):
    kind = kind.lower()
    if kind == "bar":
        chart = alt.Chart(df).mark_bar()
    elif kind == "scatter":
        chart = alt.Chart(df).mark_point()
    else:
        chart = alt.Chart(df).mark_line()

    viz = chart.encode(
        x=x,
        y=y,
        tooltip=list(df.columns)
    ).properties(width=700, height=400)
    return viz

In [18]:
df = pd.DataFrame({
    "month": ["Jan", "Feb", "Mar", "Apr"],
    "sales": [100, 120, 90, 150]
})

simplePlot(df, "month", "sales", kind="line")

alt.Chart(...)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


---
SQL Generator
---

In [20]:
import pymysql
from pymysql.cursors import DictCursor

question = "" # put a question here :)

mydb = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    database="mydb",
    port=3307,
    unix_socket="/Applications/XAMPP/xamppfiles/var/mysql/mysql2.sock",
    cursorclass=DictCursor
)

cursor = mydb.cursor()

prompt1 = f"""
You are a SQL expert assistant for a business management database.
Your task is to convert the following natural-language question into a valid, executable MySQL SQL query:

"{question}"

The database schema is as follows:

- departments(department_id, name)
- employees(employee_id, department_id, manager_id, first_name, last_name, email, phone, hire_date, salary)
- customers(customer_id, name, email, phone, billing_address, shipping_address, created_at)
- suppliers(supplier_id, name, email, phone, address)
- categories(category_id, parent_id, name, description)
- products(product_id, category_id, supplier_id, sku, name, description, unit_price, cost_price, reorder_level, is_active)
- warehouses(warehouse_id, name, address)
- inventory(product_id, warehouse_id, quantity, safety_stock)
- purchase_orders(po_id, supplier_id, employee_id, warehouse_id, order_date, expected_date, status, notes)
- purchase_order_items(poi_id, po_id, product_id, qty_ordered, qty_received, unit_cost)
- sales_orders(so_id, customer_id, employee_id, order_date, status, total_amount, shipping_address, billing_address)
- sales_order_items(soi_id, so_id, product_id, quantity, unit_price, discount)
- payments(payment_id, so_id, payment_date, amount, method, reference)
- shipments(shipment_id, so_id, warehouse_id, ship_date, carrier, tracking_number)
- returns(return_id, so_id, customer_id, product_id, return_date, quantity, reason, refund_amount)
- v_product_stock(product_id, sku, product_name, total_qty)
- v_customer_ltv(customer_id, customer_name, lifetime_value, orders_count)
- v_open_purchase_orders(po_id, supplier_name, order_date, status, qty_remaining)

Rules:
1. Output only the SQL query — no explanation, markdown, or comments.
2. Use proper JOINs based on table relationships.
3. Assume MySQL 8.0+ (supports CTEs, window functions, and DATE_FORMAT).
4. Use readable column aliases when appropriate.
5. Include WHERE, GROUP BY, and ORDER BY when logically implied.
6. When unclear which columns to show, choose the most relevant human-readable ones (names, totals, dates, etc.).
7. If the question implies a time range like “this month” or “this year,” use `CURDATE()` and MySQL date functions.

Example Input:
"Show all suppliers who provided products that are currently low in stock."

Example Output:
SELECT DISTINCT s.name AS supplier_name, p.name AS product_name, v.total_qty
FROM suppliers s
JOIN products p ON p.supplier_id = s.supplier_id
JOIN v_product_stock v ON v.product_id = p.product_id
WHERE v.total_qty <= p.reorder_level;

Example Input:
"Show total revenue per month this year."

Example Output:
SELECT DATE_FORMAT(order_date, '%Y-%m') AS month, SUM(total_amount) AS total_revenue
FROM sales_orders
WHERE YEAR(order_date) = YEAR(CURDATE()) AND status IN ('PAID','FULFILLED')
GROUP BY DATE_FORMAT(order_date, '%Y-%m')
ORDER BY month;
"""

response1 = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt1}]
)

query = response1.choices[0].message.content

cursor.execute(query)
result = cursor.fetchone()

prompt2 = f"""
You are a SQL expert assistant for this archery database. The user has asked a question and you have generated a SQL query to answer it.
The question was: "{question}"
The SQL query was: "{query}"
The result of the query is: "{result}"
present the result with simple facts.
"""

response2 = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt2}]
)

answer = response2.choices[0].message.content


ModuleNotFoundError: No module named 'pymysql'

---
SQL Data to PDF
---

---
Modular Agent Development
---

This is the tool registry that specifys what tools are avaliable for the agent

below are the tools functions

In [ ]:
def getDataID(location):
  mapsParameters = {
    "engine": "google_maps",
    "q": location,
    "type": "search",
    "api_key": SERPAPIKEY
  }

  mapsSearch = GoogleSearch(mapsParameters)
  mapsResults = mapsSearch.get_dict()

  dataId = mapsResults.get("place_results").get("data_id")

  return dataId

In [ ]:
print(getDataID("Swinburne University of Technology"))

In [ ]:
def getReviews(dataId):
  reviewParameters = {
      "engine": "google_maps_reviews",
      "data_id": dataId,
      "api_key": SERPAPIKEY,
      "hl": "en"
  }

  reviewSearch = GoogleSearch(reviewParameters)
  reviewResults = reviewSearch.get_dict()

  print(reviewResults)

  reviewTexts = []

  for review in reviewResults.get("reviews", []):
    reviewTexts.append(review["snippet"])

  return reviewTexts

In [ ]:
print(getReviews("0x6ad642326bae5aaf:0x75e96bbd4988f769"))

In [ ]:
def aspectExtraction(texts):
  aspect_extractor = ATEPC.AspectExtractor(
    'multilingual',
    auto_device=True,
    cal_perplexity=True
  )

  results = aspect_extractor.predict(
    texts,
    print_result=False,
    save_result=False,
    ignore_error=True,
    pred_sentiment=True
  )

  aspectSentiments = []

  for result in results:
    aspects = result['aspect']
    sentiments = result['sentiment']
    confidences = result['confidence']
    sentence = result['sentence']

    for i in range(len(aspects)):
      if confidences[i] > CONFIDENCETHRESHOLD:
        aspectSentiments.append({
            "aspect": aspects[i],
            "sentiment": sentiments[i],
            "sentence": sentence
        })

  return aspectSentiments

In [ ]:
TOOLREGISTRY = {
  "googleDataIdScraper": {
    "function": getDataID,
    "description": "Accesses google maps with text and returns the dataid of the top result",
    "inputs": {"location": "string"},
    "outputs": {"dataId": "string"},
  },
  "googleReviewScraper": {
    "function": getReviews,
    "description": "Accesses google reviews for a location and returns 8 results",
    "inputs": {"dataId": "string"},
    "outputs": {"reviews": "list<string>"},
  },
  "sentimentAnalysis": {
    "function": aspectExtraction,
    "description": "Performs sentiment analysis on texts and returns aspect based sentiment",
    "inputs": {"texts": "list<string>"},
    "outputs": {"list of dicts with the keys, aspect: string, sentiment: string, and sentence: string"},
  },
}

below is entering an example promps into Llama and recieving a workflow out of it

In [ ]:
def runGpt(systemPrompt, userPrompt, model):
  response = client.chat.completions.create(
    model=model,
    messages=[
      {"role": "system", "content": systemPrompt},
      {"role": "user", "content": userPrompt}
    ]
  )
  return response.choices[0].message.content

In [ ]:
# prompt engineering tweaking assisted with chatGPT

TOOLSPROMPT = """you are an AI that uses tools and you have access to the following tools:
  1. googleDataIdScraper
    - Description: Accesses Google Maps with text and returns the dataId of the top result.
    - Inputs:
        - location (string, required)
    - Outputs:
        - dataId (string)

  2. googleReviewScraper
    - Description: Accesses Google Reviews for a location and returns 8 results.
    - Inputs:
        - dataId (string, required)
    - Outputs:
        - reviews (list of strings)

  3. sentimentAnalysis
    - Description: Performs aspect-based sentiment analysis on texts.
    - Inputs:
        - texts (list of strings, required)
    - Outputs:
        - results (list of dicts with keys: aspect (string), sentiment (string), sentence (string))

  Before responding, **determine all the tools needed to answer the user’s query**.
  If more than one tool is needed, include them **in order**, so that tools that depend on previous tool outputs come **later in the list**.

  Additionally when there is a input that requires data from a previous tool, please include the tool but do not include the input

  When you need to use a tool, respond ONLY IN JSON using the following format

  [
    {
      "tool": "tool_name",
      "arguments": {
          "input_name_1": "value1",
          "input_name_2": "value2"
      }
    },
        {
      "tool": "tool_name",
      "arguments": {
          "input_name_1": "value1",
          "input_name_2": "value2"
      }
    }
  ]

  respond only in JSON and do not use any additional text
  """

In [ ]:
result = runGpt(TOOLSPROMPT, "What are some reviews of Swinburne University of Technology", llmModel)
print(result)
plan = json.loads(result)
print(plan)

In [ ]:
results = {}

for step in plan:
  toolName = step["tool"]
  toolReg = TOOLREGISTRY[toolName]
  arguments = step.get("arguments", {})

  for inputName in toolReg["inputs"]:
    if inputName not in arguments:
      found = False
      for prevTool, prevOutput in results.items():
        if inputName in prevOutput:
          arguments[inputName] = prevOutput[inputName]
          found = True
          break

  output_value = TOOLREGISTRY[toolName]["function"](**arguments)

  if isinstance(toolReg['outputs'], dict):
      output = toolReg['outputs']
  else:
      output = {toolReg['outputs']: output_value}

  results[toolName] = output


print(results)


In [ ]:
print(results['googleReviewScraper']['reviews'][1][0])